In [1]:
!pip3 install tweepy --upgrade

     |████████████████████████████████| 77 kB 2.7 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
import pickle
from keras.models import load_model    

In [3]:
import tweepy

consumer_key = "INSERTYOURCONSUMERKEYHERE"
consumer_secret = "INSERTYOURCONSUMERSECTRETHERE"
access_key = "INSERTYOURACCESSKEYHERE"
access_secret = "INSERTYOURACCESSSECRETHERE"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [7]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from geopy.geocoders import Nominatim
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import math
import re
import pandas as pd
import numpy as np
import random
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [5]:
def preprocess_text(x):
  x = ''.join([i if ord(i) < 128 else ' ' for i in x])
  x = re.sub(r'http\S+', '', x)
  x = x.replace('\n','')
  x = ' '.join(x.split())
  x = x.replace('#','')
  x = x.replace('@','')
  return x

In [6]:
def hashtag_comparison(term1,term2,num_tweets=10):
  tweets1 = tweepy.Cursor(api.search_tweets,term1+" -filter:retweets", lang="en",tweet_mode="extended").items(num_tweets)
  tweets2 = tweepy.Cursor(api.search_tweets,term2+" -filter:retweets", lang="en",tweet_mode="extended").items(num_tweets)
  list_tweets1 = [tweet for tweet in tweets1]
  list_tweets2 = [tweet for tweet in tweets2]
  text1 = []
  text2 = []
  year1 = []
  year2 = []
  month1 = []
  month2 = []
  datetime1 = []
  day1 = []
  day2 = []
  hour1 = []
  hour2 = []
  minute1 = []
  minute2 = []
  datetime2 = []
  for i in list_tweets1:
    text1.append(i.full_text)
    year1.append(i.created_at.year)
    month1.append(i.created_at.month)
    day1.append(i.created_at.day)
    hour1.append(i.created_at.hour)
    minute1.append(i.created_at.minute)
    datetime1.append(i.created_at)
  for i in list_tweets2:
    text2.append(i.full_text)
    year2.append(i.created_at.year)
    month2.append(i.created_at.month)
    day2.append(i.created_at.day)
    hour2.append(i.created_at.hour)
    minute2.append(i.created_at.minute)
    datetime2.append(i.created_at)
  df = pd.DataFrame({'Tweets1':text1,'Year1':year1,'Month1':month1,'Day1':day1,'Hour1':hour1,'minute1':minute1,'datetime1':datetime1,'Tweets2':text2,'Year2':year2,'Month2':month2,'Day2':day2,'Hour2':hour2,'minute2':minute2,'datetime2':datetime2})
  df['Tweets1'] = df['Tweets1'].apply(preprocess_text)
  df['Tweets2'] = df['Tweets2'].apply(preprocess_text)
  return df

In [8]:
df = hashtag_comparison('bullshit','terrorism',100)

In [12]:
model = load_model('my_model.h5')
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [13]:
def istoxic(x):
  if x[0]>=0.8 or x[1]>=0.8 or x[2]>=0.8 or x[3]>=0.8 or x[4]>=0.8 or x[5]>=0.8:
    return 1
  return 0

In [14]:
def plothashtags(df,h1,h2):
  maxlen = 200
  list_tokenized_test = tokenizer.texts_to_sequences(df['Tweets1'])
  X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
  to = []
  st = []
  ob = []
  thre = []
  ins = []
  ih = []
  for i in range(len(X_te)):
    pred = model.predict(X_te[i:i+1])[0]
    to.append(pred[0])
    st.append(pred[1])
    ob.append(pred[2])
    thre.append(pred[3])
    ins.append(pred[4])
    ih.append(pred[5])
  list_tokenized_test = tokenizer.texts_to_sequences(df['Tweets2'])
  X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
  to2 = []
  st2 = []
  ob2 = []
  thre2 = []
  ins2 = []
  ih2 = []
  for i in range(len(X_te)):
    pred = model.predict(X_te[i:i+1])[0]
    to2.append(pred[0])
    st2.append(pred[1])
    ob2.append(pred[2])
    thre2.append(pred[3])
    ins2.append(pred[4])
    ih2.append(pred[5])
  df['toxic1'] = to
  df['obscene1'] = ob
  df['threat1'] = thre
  df['severe_toxic1'] = st 
  df['identity_hate1'] = ih
  df['insult1'] = ins
  df['toxic2'] = to2
  df['obscene2'] = ob2
  df['threat2'] = thre2
  df['severe_toxic2'] = st2 
  df['identity_hate2'] = ih2
  df['insult2'] = ins2
  
  fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "pie"}, {"type": "pie"}]],subplot_titles=("Toxic vs Non Toxic Comments of #"+h1, "Toxic vs Non Toxic Comments of #"+h2)
  )
  df['isToxic1'] = df[['toxic1','severe_toxic1','threat1','identity_hate1','obscene1','insult1']].apply(istoxic,axis=1)
  df['isToxic2'] = df[['toxic2','severe_toxic2','threat2','identity_hate2','obscene2','insult2']].apply(istoxic,axis=1)
  df['combinedtime1'] = df['Year1'].astype(str)+'/'+df['Month1'].astype(str)+'/'+df['Day1'].astype(str)+' '+df['Hour1'].astype(str)
  df['combinedtime2'] = df['Year2'].astype(str)+'/'+df['Month2'].astype(str)+'/'+df['Day2'].astype(str)+' '+df['Hour2'].astype(str)
  di1 = pd.DataFrame({'count':[df[df['isToxic1']==0].shape[0],df[df['isToxic1']==1].shape[0]],'isToxic':['Non Toxic','Toxic']})
  di2 = pd.DataFrame({'count':[df[df['isToxic2']==0].shape[0],df[df['isToxic2']==1].shape[0]],'isToxic':['Non Toxic','Toxic']})
  fig.add_trace(
    go.Pie(values=di1['count'],labels=di1['isToxic']),
    row=1, col=1
  )
  fig.add_trace(
    go.Pie(values=di2['count'],labels=di2['isToxic']),
    row=1, col=2
  )
  fig.show()
  ###toxic, severe etc. show
  di1 = pd.DataFrame({'count':[df[df['toxic1']>=0.8].shape[0],df[df['severe_toxic1']>=0.8].shape[0],df[df['threat1']>=0.8].shape[0],df[df['identity_hate1']>=0.8].shape[0],df[df['obscene1']>=0.8].shape[0],df[df['insult1']>=0.8].shape[0],df[df['toxic2']>=0.8].shape[0],df[df['severe_toxic2']>=0.8].shape[0],df[df['threat2']>=0.8].shape[0],df[df['identity_hate2']>=0.8].shape[0],df[df['obscene2']>=0.8].shape[0],df[df['insult2']>=0.8].shape[0]],'label':['Toxic','Severe Toxic','Threat','Identity Hate','Obscene','Insult','Toxic','Severe Toxic','Threat','Identity Hate','Obscene','Insult'],'hashtag':[h1,h1,h1,h1,h1,h1,h2,h2,h2,h2,h2,h2]})
  di2 = pd.DataFrame({'count':[df[df['toxic2']>=0.8].shape[0],df[df['severe_toxic2']>=0.8].shape[0],df[df['threat2']>=0.8].shape[0],df[df['identity_hate2']>=0.8].shape[0],df[df['obscene2']>=0.8].shape[0],df[df['insult2']>=0.8].shape[0]],'label':['Toxic','Severe Toxic','Threat','Identity Hate','Obscene','Insult']})
  fig = px.bar(di1, x='label', y='count',title="Count of toxic classes",color="hashtag",barmode="group")
  fig.show()
  li1 = list(df['combinedtime1'].values)
  li2 = list(df['combinedtime2'].values)
  li1.extend(li2)
  li3 = list(df['isToxic1'].values)
  li4 = list(df['isToxic2'].values)
  li3.extend(li4)
  li5 = [h1]*df.shape[0]
  li6 = [h2]*df.shape[0]
  li5.extend(li6)
  newdf = pd.DataFrame({'time':li1,'toxic':li3,'hashtag':li5})
  newdf = newdf.groupby(['time','hashtag'])['toxic'].sum().to_frame()
  newdf.reset_index(inplace=True)
  fig = px.line(newdf, x='time', y="toxic",color='hashtag')
  fig.show()
  id = df[['toxic1']].idxmax()[0]
  if(df.loc[id,'toxic1']>=0.8):
    print('Most Toxic Comment:',df.loc[id,'Tweets1'])
    print('Toxic %:',df.loc[id,'toxic1'])
  id = df[['severe_toxic1']].idxmax()[0]
  if(df.loc[id,'severe_toxic1']>=0.8):
    print('Most Toxic Comment:',df.loc[id,'Tweets1'])
    print('Toxic %:',df.loc[id,'severe_toxic1'])
  id = df[['threat1']].idxmax()[0]
  if(df.loc[id,'threat1']>=0.8):
    print('Most Toxic Comment:',df.loc[id,'Tweets1'])
    print('Toxic %:',df.loc[id,'threat1'])
  id = df[['identity_hate1']].idxmax()[0]
  if(df.loc[id,'identity_hate1']>=0.8):
    print('Most Toxic Comment:',df.loc[id,'Tweets1'])
    print('Toxic %:',df.loc[id,'identity_hate1'])
  id = df[['obscene1']].idxmax()[0]
  if(df.loc[id,'obscene1']>=0.8):
    print('Most Toxic Comment:',df.loc[id,'Tweets1'])
    print('Toxic %:',df.loc[id,'obscene1'])
  id = df[['insult1']].idxmax()[0]
  if(df.loc[id,'insult1']>=0.8):
    print('Most Toxic Comment:',df.loc[id,'Tweets1'])
    print('Toxic %:',df.loc[id,'insult1'])
  id = df[['toxic2']].idxmax()[0]
  if(df.loc[id,'toxic2']>=0.8):
    print('Most Toxic Comment:',df.loc[id,'Tweets2'])
    print('Toxic %:',df.loc[id,'toxic2'])
  id = df[['severe_toxic2']].idxmax()[0]
  if(df.loc[id,'severe_toxic2']>=0.8):
    print('Most Toxic Comment:',df.loc[id,'Tweets2'])
    print('Toxic %:',df.loc[id,'severe_toxic2'])
  id = df[['threat2']].idxmax()[0]
  if(df.loc[id,'threat2']>=0.8):
    print('Most Toxic Comment:',df.loc[id,'Tweets2'])
    print('Toxic %:',df.loc[id,'threat2'])
  id = df[['identity_hate2']].idxmax()[0]
  if(df.loc[id,'identity_hate2']>=0.8):
    print('Most Toxic Comment:',df.loc[id,'Tweets2'])
    print('Toxic %:',df.loc[id,'identity_hate2'])
  id = df[['obscene2']].idxmax()[0]
  if(df.loc[id,'obscene2']>=0.8):
    print('Most Toxic Comment:',df.loc[id,'Tweets2'])
    print('Toxic %:',df.loc[id,'obscene2'])
  id = df[['insult2']].idxmax()[0]
  if(df.loc[id,'insult2']>=0.8):
    print('Most Toxic Comment:',df.loc[id,'Tweets2'])
    print('Toxic %:',df.loc[id,'insult2'])

  return df

In [17]:
def geotweets(lat,long,hashtag="",km=1000,num_tweets=50):
  tweets = tweepy.Cursor(api.search_tweets,hashtag+" -filter:retweets",geocode =str(lat)+','+str(long)+','+str(km)+'km', lang="en",tweet_mode="extended").items(num_tweets)
  list_tweets = [tweet for tweet in tweets]
  locator = Nominatim(user_agent="rishabh21072@iiitd.ac.in")
  text1 = []
  year1 = []
  month1 = []
  day1 = []
  hour1 = []
  minute1 = []
  datetime1 = []
  location1 = []
  for i in list_tweets:
    text1.append(i.full_text)
    year1.append(i.created_at.year)
    month1.append(i.created_at.month)
    day1.append(i.created_at.day)
    hour1.append(i.created_at.hour)
    minute1.append(i.created_at.minute)
    datetime1.append(i.created_at)
    location1.append(i.user.location)
  df = pd.DataFrame({'text':text1,'year':year1,'month':month1,'day':day1,'hour':hour1,'minute':minute1,'datetime':datetime1,'location':location1})
  df['latitude']=0
  df['longitude'] = 0
  for i in range(df.shape[0]):
    try:
      location = locator.geocode(df.iloc[i,7])
      df.loc[i,'latitude'] = location.latitude
      df.loc[i,'longitude'] = location.longitude
    except:
      continue
  df['text'] = df['text'].apply(preprocess_text)
  return df

In [18]:
df2 = geotweets(20.5937,78.9629,'bullshit')

In [19]:
df2['latitude'] = df2['latitude'].astype(float)
df2['longitude'] = df2['longitude'].astype(float)
print('Total number of tweets extracted shown on map')
m_5 = folium.Map(location=[20.5937,78.9629], tiles='cartodbpositron', zoom_start=5)
mc = MarkerCluster()
# Add a heatmap to the base map
for idx, row in df2.iterrows():
    if not math.isnan(row['longitude']) and not math.isnan(row['latitude']):
        mc.add_child(Marker([row['latitude'], row['longitude']],popup=row['text']))
# Display the map
m_5.add_child(mc)
m_5

Total number of tweets extracted shown on map


In [20]:
df2['latitude'] = df2['latitude'].astype(float)
df2['longitude'] = df2['longitude'].astype(float)
#####predict
maxlen = 200
list_tokenized_test = tokenizer.texts_to_sequences(df2['text'])
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
to = []
st = []
ob = []
thre = []
ins = []
ih = []
for i in range(len(X_te)):
  pred = model.predict(X_te[i:i+1])[0]
  to.append(pred[0])
  st.append(pred[1])
  ob.append(pred[2])
  thre.append(pred[3])
  ins.append(pred[4])
  ih.append(pred[5])
df2['toxic'] = to
df2['obscene'] = st
df2['threat'] = ob
df2['severe_toxic'] = thre
df2['identity_hate'] = ins
df2['insult'] = ih
#####predict end
df2['isToxic'] = df2[['toxic','severe_toxic','threat','identity_hate','obscene','insult']].apply(istoxic,axis=1)
print('Total number of TOXIC tweets extracted shown on map')
m_5 = folium.Map(location=[20.5937,78.9629], tiles='cartodbpositron', zoom_start=5)
mc = MarkerCluster()
# Add a heatmap to the base map
for idx, row in df2[df2['isToxic']==1].iterrows():
    if not math.isnan(row['longitude']) and not math.isnan(row['latitude']):
        mc.add_child(Marker([row['latitude'], row['longitude']],popup=row['text']))
# Display the map
m_5.add_child(mc)
m_5

Total number of TOXIC tweets extracted shown on map


In [15]:
dfxx = plothashtags(df.copy(),'bullshit','terrorism')

Most Toxic Comment: Fuck ICA make what stupid arrival card bullshit.
Toxic %: 0.9990906
Most Toxic Comment: Fuck ICA make what stupid arrival card bullshit.
Toxic %: 0.98694474
Most Toxic Comment: umwhatthefuck10 tommotopdollar9 jlrose13 gevus1 AteequeYaqoob emilykschrader bellahadid That's slanderously antisemitic, now you're just spewing bs. Palestinians aren't fucking treated like jews were, no camps no gas chambers no stars no experiments no rapes wtf are you on, palestinians have a limitation of movement due to terrorism, jews had hell for being jews
Toxic %: 0.98836136
Most Toxic Comment: umwhatthefuck10 tommotopdollar9 jlrose13 gevus1 AteequeYaqoob emilykschrader bellahadid That's slanderously antisemitic, now you're just spewing bs. Palestinians aren't fucking treated like jews were, no camps no gas chambers no stars no experiments no rapes wtf are you on, palestinians have a limitation of movement due to terrorism, jews had hell for being jews
Toxic %: 0.9056726
